# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [5]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [12]:
total = data.groupby(by = ['CustomerID', 'ProductName'])['Quantity'].sum()
total = pd.DataFrame(total.reset_index())
total

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [7]:
pivot_data = pd.pivot_table(data, index= 'ProductName', values = 'Quantity', columns = 'CustomerID').fillna(value = 0)

In [8]:
pivot_data.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [9]:
pdist_data = squareform(pdist(pivot_data, metric = 'euclidean'))

In [10]:
pdist_data

array([[  0.        , 239.16312425, 234.44615587, ..., 232.91629398,
        235.03829475, 235.99788135],
       [239.16312425,   0.        , 237.88232385, ..., 230.52765561,
        240.83189158, 235.95338523],
       [234.44615587, 237.88232385,   0.        , ..., 237.00421937,
        235.64379898, 236.82060721],
       ...,
       [232.91629398, 230.52765561, 237.00421937, ...,   0.        ,
        246.73264883, 238.5644567 ],
       [235.03829475, 240.83189158, 235.64379898, ..., 246.73264883,
          0.        , 232.03878986],
       [235.99788135, 235.95338523, 236.82060721, ..., 238.5644567 ,
        232.03878986,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
distances = pd.DataFrame(((1 + squareform(pdist(pivot_data.T, 'euclidean')))**-1),columns=pivot_data.columns,index=pivot_data.columns)
top5_customer = list((distances[33].sort_values(ascending=False).reset_index()).iloc[1:6,0])
top5_customer

[3909, 264, 3531, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [24]:
top5_product = total.loc[(total['CustomerID'].isin(top5_customer)),:]
top5_product

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [23]:
top5_product_sum = top5_product.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
top5_product_sum

ProductName
Salsify, Organic                 4
Wine - Charddonnay Errazuriz     3
Wine - Ej Gallo Sierra Valley    3
Oranges - Navel, 72              3
Quiche Assorted                  3
                                ..
Pepsi - Diet, 355 Ml             1
Pepper - White, Ground           1
Peas - Pigeon, Dry               1
Pastry - Cheese Baked Scones     1
Appetizer - Mushroom Tart        1
Name: Quantity, Length: 235, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [20]:
df_7 = pd.merge(left=top5_product_sum, right=pivot_data, on='ProductName')
df_7.loc[(df_7[33] == 0),:].sort_values(by='Quantity', ascending=False)['Quantity'][0:5]

ProductName
Wine - Charddonnay Errazuriz    3
Oranges - Navel, 72             3
Bay Leaf                        3
Pecan Raisin - Tarts            3
Chocolate - Dark                3
Name: Quantity, dtype: int64

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [26]:
recommendations = {}
for user in data.CustomerID.unique():
    top5_customer = list((distances[user].sort_values(ascending=False).reset_index()).iloc[1:6,0])
    top5_product = total.loc[(total['CustomerID'].isin(top5_customer)),:]
    top5_product_sum = top5_product.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    df_7 = pd.merge(left=top5_product_sum, right=pivot_data, on='ProductName')
    rec = list((df_7.loc[(df_7[33] == 0),:].sort_values(by='Quantity', ascending=False)['Quantity'][0:5]).index)
    recommendations[user]=rec

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [27]:
df_recommendations = pd.DataFrame.from_dict(recommendations, orient='index', columns=['Prod1','Prod2','Prod3','Prod4','Prod5'])
df_recommendations

,Prod1,Prod2,Prod3,Prod4,Prod5
61288,"Mushrooms - Black, Dried",Sponge Cake Mix - Chocolate,"Wine - Magnotta, Merlot Sr Vqa",Beef - Rib Eye Aaa,Chicken - Soup Base
77352,Guinea Fowl,Grenadine,"Oranges - Navel, 72",Ecolab - Mikroklene 4/4 L,"Beets - Candy Cane, Organic"
40094,"Water - Mineral, Natural",Pasta - Orecchiette,Wiberg Super Cure,"Oregano - Dry, Rubbed",Clam Nectar
23548,Tea - English Breakfast,Spinach - Baby,Wanton Wrap,Banana Turning,Pie Filling - Cherry
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Crush - Cream Soda,Beef - Inside Round
...,...,...,...,...,...
49005,Olive - Spread Tapenade,Macaroons - Two Bite Choc,Mayonnaise - Individual Pkg,Spinach - Baby,Cumin - Whole
41286,Veal - Inside,Beef - Ground Medium,Guinea Fowl,Chips Potato All Dressed - 43g,Tea - Earl Grey
85878,Kellogs All Bran Bars,Cookie - Dough Variety,"Beef - Ground, Extra Lean, Fresh",Muffin - Carrot Individual Wrap,Bread - Rye
68506,"Doilies - 5, Paper",Tea - Decaf Lipton,Beef - Top Sirloin - Aaa,Cocoa Butter,Beer - Sleemans Cream Ale


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [31]:

distances1 = pd.DataFrame(((1 + squareform(pdist(pivot_data.T, 'cityblock')))**-1),columns=pivot_data.columns,index=pivot_data.columns)
recommendations1 = {}
for user in data.CustomerID.unique():
    top5_customer = list((distances1[user].sort_values(ascending=False).reset_index()).iloc[1:6,0])
    top5_product = total.loc[(total['CustomerID'].isin(top5_customer)),:]
    top5_product_sum = top5_product.groupby('ProductName')['Quantity'].sum().sort_values(ascending=False)
    df_7 = pd.merge(left=top5_product_sum, right=pivot_data, on='ProductName')
    rec = list((df_7.loc[(df_7[33] == 0),:].sort_values(by='Quantity', ascending=False)['Quantity'][0:5]).index)
    recommendations1[user]=rec
df_recommendations1 = pd.DataFrame.from_dict(recommendations1, orient='index', columns=['Prod1','Prod2','Prod3','Prod4','Prod5'])
df_recommendations1

,Prod1,Prod2,Prod3,Prod4,Prod5
61288,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Ice Cream Bar - Hageen Daz To,Bay Leaf,Wine - Ej Gallo Sierra Valley
77352,Soup - Campbells Tomato Ravioli,Pomello,"Cheese - Brie, Triple Creme",Cheese - Camembert,Bread Crumbs - Panko
40094,Butter - Unsalted,Cod - Black Whole Fillet,Crush - Cream Soda,Tea - Decaf Lipton,Wine - Ej Gallo Sierra Valley
23548,Soup - Campbells Tomato Ravioli,Muffin - Carrot Individual Wrap,Pork - Kidney,"Sole - Dover, Whole, Fresh",Puree - Passion Fruit
78981,Watercress,Cumin - Whole,Beef - Montreal Smoked Brisket,Butter - Unsalted,Tia Maria
...,...,...,...,...,...
49005,Knife Plastic - White,Tofu - Firm,Otomegusa Dashi Konbu,Bay Leaf,Butter - Unsalted
41286,"Cheese - Boursin, Garlic / Herbs",Bar - Granola Trail Mix Fruit Nut,Sausage - Breakfast,Ice Cream Bar - Hageen Daz To,Yoghurt Tubes
85878,Juice - Lime,Lime Cordial - Roses,Bar - Granola Trail Mix Fruit Nut,Soup - Campbells Tomato Ravioli,"Spoon - Soup, Plastic"
68506,Ice Cream Bar - Hageen Daz To,Soup - Campbells Tomato Ravioli,Pork - Kidney,Halibut - Steaks,Wine - Magnotta - Belpaese
